In [ ]:
pip install pandas
pip install matplotlib
pip install seaborn

: 

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
from matplotlib.gridspec import GridSpec
import seaborn as sns
import re

In [ ]:
df = pd.read_csv('../data/raw_data.csv', index_col= 0)
df.head(10)

In [ ]:
df_renamed = df.rename(columns = {"Ngày":"date", "Địa chỉ":"address", "Quận":"district", 
                                  "Huyện":"ward", "Loại hình nhà ở":"type_of_housing",
                                 "Giấy tờ pháp lý":"legal_paper", "Số tầng":"num_floors",
                                 "Số phòng ngủ":"num_bed_rooms", "Diện tích":"squared_meter_area",
                                 "Dài":"length_meter", "Rộng":"width_meter", "Giá/m2":"price_in_million_per_square_meter"})




# Filter data
# Remove houses with "10 plus" floors and bed rooms, since this cannot be exactly quantified
df_renamed = df_renamed[df_renamed['num_floors'] != 'Nhiều hơn 10']
df_renamed = df_renamed[df_renamed['num_bed_rooms'] != 'nhiều hơn 10 phòng']
df_renamed['date'] = pd.to_datetime(df_renamed['date'], errors='coerce')
# Clean columns and convert numerical columns to float type
    # Add more features ( if needed)

df_renamed['year'] = df_renamed['date'].dt.year
df_renamed['month'] = df_renamed['date'].dt.month
df_renamed['day_of_week'] = df_renamed['date'].dt.dayofweek
df_renamed['is_weekend'] = df_renamed['date'].dt.weekday >= 5

df_renamed['address'] = df_renamed['address'].fillna("UNK")
df_renamed['street'] = df_renamed['address'].str.split(',').apply(lambda x: x[0].strip())

df_renamed['district'] = df_renamed['district'].str.replace('Quận ','').str.strip()
df_renamed['ward'] = df_renamed['ward'].str.replace('Phường ','').str.strip()
df_renamed['num_floors'] = df_renamed['num_floors'].str.strip().astype(float)
df_renamed['num_bed_rooms'] = df_renamed['num_bed_rooms'].str.replace(' phòng','').str.strip().astype(float)
df_renamed['squared_meter_area'] = df_renamed['squared_meter_area'].str.replace(' m²','').str.strip().astype(float)
df_renamed['length_meter'] = df_renamed['length_meter'].str.replace(' m','').str.strip().astype(float)
df_renamed['width_meter'] = df_renamed['width_meter'].str.replace(' m','').str.strip().astype(float)



cat_col = ['address', 'street','district', 'ward', 'type_of_housing', 'legal_paper']
num_col = [col for col in df_renamed.columns if col not in cat_col]

df_renamed[cat_col] = df_renamed[cat_col].fillna("UNK")
df_renamed[num_col] = df_renamed[num_col].fillna(-1)



# Clean and convert all prices to million/m2 instead of VND/m2 or billion/m2
        # Convert values containing 'tỷ/m²'
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' tỷ/m²', na=False), 'price_in_million_per_square_meter'] = \
    df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' tỷ/m²', na=False), 'price_in_million_per_square_meter'] \
    .str.replace(' tỷ/m²', '').str.replace('.', '').str.replace(',', '.').astype(float) * 1000

        # Convert values containing 'triệu/m²'
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'] = \
    df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'] \
    .str.replace(' triệu/m²', '').str.replace(',', '.').astype(float)

        # Convert values containing 'đ/m²'
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' đ/m²', na=False), 'price_in_million_per_square_meter'] = \
    df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' đ/m²', na=False), 'price_in_million_per_square_meter'] \
    .str.replace(' đ/m²', '').str.replace('.', '').astype(float) * 0.000001
df_renamed['price_in_million_per_square_meter'] = df_renamed['price_in_million_per_square_meter'].astype(float)
# Add informative features
# Total number of rooms (sum of floors and bedrooms)
df_renamed['total_rooms'] = df_renamed['num_floors'] + df_renamed['num_bed_rooms']

# Floors per Bedroom (ratio of number of floors to number of bedrooms)
df_renamed['floors_per_bedroom'] = df_renamed['num_floors'] / df_renamed['num_bed_rooms']


# Remove redundant columns
df_renamed = df_renamed.drop(columns= ['address', 'date'])



In [ ]:
df_renamed.head(1)

In [ ]:
df_renamed.info()   

# EDA PART

In [ ]:
# Setting up plots
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# 1. Descriptive Statistics
numeric_summary = df_renamed.describe()
categorical_columns = df_renamed.select_dtypes(include='object').columns
df_renamed[categorical_columns].describe()



In [ ]:
# 2. Distribution of categorical variables
district_distribution = df_renamed['district'].value_counts()
housing_type_distribution = df_renamed['type_of_housing'].value_counts()
legal_docs_distribution = df_renamed['legal_paper'].value_counts()

fig, axes = plt.subplots(1, 3, figsize=(18, 10), constrained_layout=True)

# Distribution of districts
axes[0].barh(district_distribution.index, district_distribution.values, color='skyblue')
axes[0].set_title('Distribution of Districts')
axes[0].set_ylabel('Districts')
axes[0].set_xlabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Distribution of housing types
axes[1].bar(housing_type_distribution.index, housing_type_distribution.values, color='lightgreen')
axes[1].set_title('Distribution of Housing Types')
axes[1].set_xlabel('Housing Type')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

# Distribution of legal documents
axes[2].bar(legal_docs_distribution.index, legal_docs_distribution.values, color='lightcoral')
axes[2].set_title('Distribution of Legal Documents')
axes[2].set_xlabel('Legal Document Type')
axes[2].set_ylabel('Count')
axes[2].tick_params(axis='x', rotation=45)

# Show the plots
plt.show()


In [ ]:

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 3. Price Distribution
sns.histplot(df_renamed['price_in_million_per_square_meter'], bins=50, kde=True, color='blue', ax=axes[0, 0])
axes[0, 0].set_title('Distribution of Price per Square Meter (Giá/m2)')
axes[0, 0].set_xlabel('Price per Square Meter (million VND)')
axes[0, 0].set_ylabel('Frequency')

# 4. Area Distribution
sns.histplot(df_renamed['squared_meter_area'], bins=50, kde=True, color='green', ax=axes[0, 1])
axes[0, 1].set_title('Distribution of Property Area (m²)')
axes[0, 1].set_xlabel('Area (m²)')
axes[0, 1].set_ylabel('Frequency')

# 5. Relationship Between Area and Price
sns.scatterplot(x=df_renamed['squared_meter_area'], y=df_renamed['price_in_million_per_square_meter'], alpha=0.5, color='purple', ax=axes[0, 2])
axes[0, 2].set_title('Relationship Between Area and Price per Square Meter')
axes[0, 2].set_xlabel('Area (m²)')
axes[0, 2].set_ylabel('Price per Square Meter (million VND)')

# 6. Average Price by District
district_avg_price = df_renamed.groupby('district')['price_in_million_per_square_meter'].mean().sort_values(ascending=False)
district_avg_price.plot(kind='barh', color='coral', edgecolor='black', ax=axes[1, 0])
axes[1, 0].set_title('Average Price per Square Meter by District')
axes[1, 0].set_xlabel('District')
axes[1, 0].set_ylabel('Average Price (million VND)')

axes[1, 1].axis('off')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()


# data có quá nhiều outliers dẫn dến distribution qúa lệch -> nên xử lí outliers

In [ ]:
# 7. Correlation Heatmap
correlation_matrix = df_renamed[[col for col in df_renamed.columns if col not in categorical_columns]].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')